<a href="https://colab.research.google.com/github/michalis0/Cloud-and-Advanced-Analytics/blob/main/labs/2-BigQuery/notebooks/BigQuery_exercises_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google BigQuery

## Walkthrough

### Authentication

In [ ]:
import os

PROJECT_NAME = "" # REPLACE WITH YOUR PROJECT NAME


#### For Colab users (recommended)

In [ ]:
from google.colab import auth
auth.authenticate_user()
print("Authenticated")

#### For Jupyter users ONLY (more challenging)

Follow [this](https://cloud.google.com/iam/docs/creating-managing-service-account-keys) guide to create the JSON with the service account key. Then, make sure to replace "PATH_TO_CREDENTIALS_FILE" with the *absolute* path to the JSON service account key (e.g., "C:/Users/John/credentials.json").

In [ ]:
# %pip install google-cloud-bigquery
# %pip install db-dtypes

In [ ]:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = PATH_TO_KEY

### Connect to BigQuery

In [ ]:
import pandas as pd
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(project=PROJECT_NAME)

To run the cells below, you must first create a new dataset in BigQuery and upload the data from week 2 (`Teams.csv` and `Players.csv`) as tables. Since the CSV files are well-formed, you can automatically infer the schema when loading the data.

In [ ]:
# Create a reference to the World Cup dataset
dataset_ref = client.dataset("world_cup", project=PROJECT_NAME)  # CHANGE WITH YOUR OWN DATASET (NOT TABLE) NAME AND PROJECT ID

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [ ]:
# List the tables in the dataset
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

In [ ]:
# Create a reference to the "Teams" table and fetch the table
table_ref = dataset_ref.table("teams") # Make sure it matches the table name printed above (case sensitive)
teams_table = client.get_table(table_ref)

# Same for the "Players" table
table_ref = dataset_ref.table("players") # Make sure it matches the table name printed above (case sensitive)
players_table = client.get_table(table_ref)

In [ ]:
# Display the schema of the "Teams" table
players_table.schema

In [ ]:
# Show a preview of the "Teams" table using pandas
client.list_rows(teams_table, max_results=5).to_dataframe()

### Example of queries

In [ ]:
q1 = """
select *
from %s.world_cup.teams
where ranking < 20
order by ranking
""" %PROJECT_NAME

query_job = client.query(q1)
query_job.to_dataframe()

In [ ]:
# Join the two tables
q2 = """
select teams.*, players.surname, players.position, players.shots,
players.minutes, players.passes, players.tackles, players.saves
from %s.world_cup.teams teams
join %s.world_cup.players players
on teams.team = players.team
""" %(PROJECT_NAME, PROJECT_NAME)

query_job = client.query(q2)
query_job.to_dataframe().head(10)

### Basic Queries

In [ ]:
query = "SELECT * FROM {}.world_cup.players LIMIT 5".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

In [ ]:
query = "SELECT * FROM {}.world_cup.teams LIMIT 5".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

In [ ]:
query = """SELECT surname, team, minutes, passes
            FROM {}.world_cup.players
            WHERE team="Switzerland" AND surname LIKE "%er%"
            ORDER BY surname""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

In [ ]:
query = """SELECT team, COUNT(*) AS number_players_listed
            FROM {}.world_cup.players
            GROUP BY team
            HAVING number_players_listed<20
            ORDER BY number_players_listed DESC
            LIMIT 5""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

In [ ]:
query = """SELECT surname, position, minutes, T.team, ranking, games, wins, draws, losses
            FROM {}.world_cup.players as P
            JOIN {}.world_cup.teams AS T
            ON P.team=T.team
            WHERE surname <"C"
            LIMIT 5""".format(PROJECT_NAME, PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

## Exercise 1 - World Cup datasets

### Simple Queries
The first three questions are already solved for you, so that you have concrete examples of queries. Try to solve the remaining ones!

*1)  Which player on a team ending with "ia" played less than 200 minutes and made more than 100 passes? Return the player's surname and team.*

**Hint**: To check if attribute A contains a (sub)string S, use the LIKE keyword (e.g. `A like '%S%'`). The % sign indicates a wildcard.

In [ ]:
# Your query goes here
query = """select surname, team
            from {}.world_cup.players as P
            where P.team like '%ia'
            and P.minutes < 200
            and P.passes > 100""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*2) Find all players who made more than 20 shots. Return all player information in descending order of shots made.*

**Hint**: Sorting results is done via the ORDER BY keyword. The default order is ascending (ASC). If you want descending order, use DESC (e.g. `ORDER BY column_1, column_2 DESC`).

In [ ]:
# Your query goes here
query = """select *
            from {}.world_cup.players as P
            where shots > 20
            order by shots desc""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*3) Which team has the highest average number of passes per minute played? Return the team's name and average number of passes per minute.*

**Hint #1**: You can compute a team's average number of passes per minute played by dividing the total number of passes by the total number of minutes. To force floating point division, multiply one operand by 1.0.

**Hint #2**: Consider using the LIMIT keyword.

In [ ]:
# Your query goes here
query = """SELECT *, (1.0 * total_passes / total_minutes) AS avg_ppm
            FROM (
                SELECT team, sum(passes) AS total_passes, sum(minutes) AS total_minutes
                FROM {}.world_cup.players_and_teams
                GROUP BY team)
            ORDER BY avg_ppm DESC
            LIMIT 1""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*4) Find the goalkeepers of teams that played more than four games. List the surname of the goalkeeper, the team, and the number of minutes the goalkeeper played.*

**Hint**: Use the `PlayersExt` table.

In [ ]:
# Your query goes here
query = """select surname, team, minutes
            FROM {}.world_cup.players_and_teams
            where position like "goal%"
            and games > 4""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*5) How many players on a team with a ranking lower than 10 played more than 350 minutes? Return a single number in a column named "superstar".*

**Hint**: To rename a column, use the AS keyword (e.g. `SELECT column_1 AS label`).

In [ ]:
# Your query goes here
query = """select count(surname) as superstar
            FROM {}.world_cup.players_and_teams
            where ranking < 10
            and minutes > 350""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*6) What is the average number of passes made by forwards? What about midfielders? Write one query that returns both values with the corresponding position.*

**Hint**: Use the GROUP BY keyword. GROUP BY statements are often used in conjuction with aggregate functions like AVG(), SUM() or COUNT().

In [ ]:
# Your query goes here
query = """select position, avg(passes) as avg_passes
            FROM {}.world_cup.players
            where position in ("forward", "midfielder")
            group by position""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

### Advanced Queries
Now, on to more challenging questions...

*1) Find all pairs of teams that have the same number of `goalsFor` as well as the same number of `goalsAgainst` as each other. Return the team pairs and their respective numbers of `goalsFor` and `goalsAgainst` (make sure to return each pair only once!).*

**Hint**: You basically need to do a "self join" of the `Teams` table. For that, you need to join different name aliases of the same table. Check [this page](https://www.w3schools.com/sql/sql_join_self.asp) for help.

In [ ]:
# Your query goes here
query = """select *
        FROM {}.world_cup.teams as T1, {}.world_cup.teams as T2
        LIMIT 3""".format(PROJECT_NAME, PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

In [ ]:
query = """select A.team AS team_A, B.team AS team_B, A.goalsFor, A.goalsAgainst
            FROM {}.world_cup.teams as A, {}.world_cup.teams as B
            where A.team < B.team
            and A.goalsFor = B.goalsFor
            and A.goalsAgainst = B.goalsAgainst""".format(PROJECT_NAME, PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*2) Find all teams with a ranking below 30 where no player has made more than 150 passes. Return the team's name and ranking.*

**Hint #1**: Consider using the HAVING keyword.

**Hint #2**: You may also want to look up nested queries.

In [ ]:
# Your query goes here
query = """ SELECT *
            FROM (
                select team, min(ranking) as ranking, max(passes) as max_passes
                from {}.world_cup.players_and_teams
                group by team
            )
            where ranking < 30
            and max_passes <= 150
            """.format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*3) Which team has the highest ratio of goalsFor to goalsAgainst?*

In [ ]:
# Your query goes here
query = """select team, goalsFor, goalsAgainst, (1.0 * goalsFor / goalsAgainst) as ratio
            FROM {}.world_cup.teams
            order by ratio desc
            limit 1""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*4) Find all teams whose defenders averaged more than 150 passes. Return the team and average number of passes by defenders, in descending order of average passes.*

In [ ]:
# Your query goes here
query = """select team, avg(passes) as avg_passes
            FROM {}.world_cup.players_and_teams
            where position like "defender"
            group by team
            having avg_passes > 150
            order by avg_passes desc""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

## Exercise 2 - Google Cloud datasets

Let's build some queries for the `accidents_2016` data from the NHTSA traffic facilities dataset, which is publicly available on Google BigQuery.

### Setting up

In [ ]:
# Create a reference to the traffic facilities dataset
traffic_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
traffic_dataset = client.get_dataset(traffic_ref)

In [ ]:
# Construct a reference to the "accident_2016" table
accidents_ref = traffic_ref.table("accident_2016")

# API request - fetch the table
accidents_table = client.get_table(accidents_ref)

In [ ]:
# Display schema
accidents_table.schema

In [ ]:
# Show a preview of the "accident_2016" table
client.list_rows(accidents_table, max_results=5).to_dataframe()

### Questions

You can access the table using `from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`.

1) How many crashes happened in each month of 2016?

In [ ]:
q3 = """
select month_of_crash, count(*) as count
from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016
group by month_of_crash
order by month_of_crash
"""

query_job_3 = client.query(q3)
query_job_3.to_dataframe()

2) Let's call the accidents involving more than 5 motor vehichles in transport involved as "massive_crash", and all others as "small crash". Return the number of each type of crash for each weather condition.

**Hint**: Use columns `atmospheric_conditions_1_name` and `number_of_motor_vehicles_in_transport_mvit`.

In [ ]:
q4 = """
select
  atmospheric_conditions_1_name,
  (case when number_of_motor_vehicles_in_transport_mvit > 5 then "massive_crash"
  else "small_crash" end) as crash_type,
  count(*) as nb_crashes
from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016
group by atmospheric_conditions_1_name, crash_type
order by atmospheric_conditions_1_name
"""

query_job_4 = client.query(q4)
query_job_4.to_dataframe()

3) Find the top 5 states with the highest number of drunk drivers involved in accidents.

In [ ]:
q5 = """
select state_name, sum(number_of_drunk_drivers) as nb_drunk_drivers
from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016
group by state_name
order by nb_drunk_drivers desc
limit 5
"""

query_job_5 = client.query(q5)
query_job_5.to_dataframe()

4) Let's find out the states where using a cell phone is a major cause of accidents. Return the first 5 state names along with the number of crashes where the driver was distracted "while manipulating cellular phone".

**Hint**: You need to join the `distract_2016` and `accident_2016` tables.

In [ ]:
q6 = """
select
  count(distract.driver_distracted_by) as nb_accidents,
  accident.state_name
from bigquery-public-data.nhtsa_traffic_fatalities.distract_2016 distract
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on distract.consecutive_number = accident.consecutive_number
where driver_distracted_by = 6
group by state_name
order by nb_accidents desc
limit 5
"""

query_job_6 = client.query(q6)
query_job_6.to_dataframe()

In [ ]:
q6_1 = """
with
  join_acc_dist as (
    select distract.driver_distracted_by, accident.state_name
    from bigquery-public-data.nhtsa_traffic_fatalities.distract_2016 distract
    join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
    on distract.consecutive_number = accident.consecutive_number
    )
select count(driver_distracted_by) as nb_accidents, state_name from join_acc_dist
where driver_distracted_by = 6
group by state_name
order by nb_accidents desc
limit 5
"""

query_job_6_1 = client.query(q6_1)
query_job_6_1.to_dataframe()

5) Sometimes, a "live animal" is a reason that a driver maneuvers and causes an accident. In which months of the year do such accidents happen the most? Return the month and the number of accidents.

**Hint**: You need to join the `maneuver_2016` and `accident_2016` tables.

In [ ]:
q7 = """
select
  accident.month_of_crash,
  count(maneuver.driver_maneuvered_to_avoid) as nb_accidents
from bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016 maneuver
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on maneuver.consecutive_number = maneuver.consecutive_number
where driver_maneuvered_to_avoid = 3
group by month_of_crash
order by nb_accidents desc
"""

query_job_7 = client.query(q7)
query_job_7.to_dataframe()

6) In the state of Alaska, in which months do "poor road conditions" cause the most accidents? Return the month and the number of accidents.

**Hint**: Again, you will need the `maneuver_2016` table.

In [ ]:
q8 = """
select
  accident.month_of_crash,
  count(maneuver.driver_maneuvered_to_avoid) as nb_accidents
from bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016 maneuver
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on maneuver.consecutive_number = maneuver.consecutive_number
where driver_maneuvered_to_avoid = 3
  and accident.state_name = "Alaska"
group by month_of_crash
order by nb_accidents desc
"""

query_job_8 = client.query(q8)
query_job_8.to_dataframe()

7) Sometimes, a factor of the accident is that the vehicle has a problem with the "brake system". Filter the accidents with this factor and return the `driver_maneuvered_to_avoid_name` column along with the related number of accidents.

**Hint**: You need to join the `maneuver_2016` and `factor_2016` tables.

In [ ]:
q9 = """
select
  maneuver.driver_maneuvered_to_avoid_name,
  count(maneuver.consecutive_number) as num_incidents
from bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016 maneuver
join bigquery-public-data.nhtsa_traffic_fatalities.factor_2016 factor
on maneuver.consecutive_number = factor.consecutive_number
where factor.contributing_circumstances_motor_vehicle = 2
group by driver_maneuvered_to_avoid_name
order by num_incidents desc
"""

query_job_9 = client.query(q9)
query_job_9.to_dataframe()

8) Sometimes, a non-motorist is involved in a crash. For each state, find the number of crashes in which an "inattentive" person was involved. Return the state name and number of crashes.

**Hint**: You need to join the `nmcrash_2016` and `accident_2016` tables.

In [ ]:
q10 = """
select
  accident.state_name,
  count(nmcrash.consecutive_number) as nb_accidents
from bigquery-public-data.nhtsa_traffic_fatalities.nmcrash_2016 nmcrash
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on nmcrash.consecutive_number = accident.consecutive_number
where nmcrash.non_motorist_contributing_circumstances = 6
group by state_name
order by nb_accidents desc
"""

query_job_10 = client.query(q10)
query_job_10.to_dataframe()